In [4]:
from sys import stdout
import pandas as pd
import numpy as np
import os

from openmm.app import *
from openmm import *
from openmm.unit import *
from pdbfixer.pdbfixer import PDBFixer, proteinResidues

In [2]:
pdb = '1QLQ' #BPTI
T_crys = 292  #Crystallization conditions are on REMARK 280 lines
pH = 7.5

In [3]:
!wget https://files.rcsb.org/view/1QLQ.pdb

--2023-03-06 06:39:55--  https://files.rcsb.org/view/1QLQ.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.210.234
Connecting to files.rcsb.org (files.rcsb.org)|132.249.210.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘1QLQ.pdb.2’

1QLQ.pdb.2              [ <=>                ] 132.89K  --.-KB/s    in 0.1s    

2023-03-06 06:39:55 (1.22 MB/s) - ‘1QLQ.pdb.2’ saved [136080]



- [Prep pdb and run a basic simulation](#prep)
- [Custom forces](#smp)

<a id = 'prep'></a>
### Preparing for MD and Running a Basic Simulation

Example function to prep a pdb file. You can find further examples in the source code for openmm-setup: https://github.com/openmm/openmm-setup/blob/master/setup.py

Define simulation parameters

In [10]:
eq_steps = 10000
prod_steps = 300000
barostatInterval = 200
pressure = 1*atmospheres
temperature = 295*kelvin
friction = 1/picosecond
timestep = 0.002*picoseconds
nonbonded_cutoff = 1*nanometer
nonbonded_method = PME
constraints = HBonds

In [11]:
pdb = PDBFile('1QLQ_processed.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3p.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod = PME,
                                 nonbondedCutoff = 1*nanometer, constraints = HBonds)
integrator = LangevinMiddleIntegrator(T_crys, friction, timestep)


In [12]:
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

In [ ]:
simulation.minimizeEnergy()
positions = simulation.context.getState(getPositions=True).getPositions()
#simulation.reporters.append(PDBReporter('1QLQ_processed.pdb', 10000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.reporters.append(StateDataReporter('./1QLQ_ProdRun.log', 10000, step=True,
                                              potentialEnergy=True, temperature=True,  progress=True,
                                              remainingTime=True, speed=True, totalSteps=prod_steps,
                                              separator='\t'))
simulation.reporters.append(app.DCDReporter('1QLQ_trajectory.dcd', 5000))
simulation.step(30000)
positions = simulation.context.getState(getPositions=True).getPositions()
PDBFile.writeFile(simulation.topology, positions, open('1QLQ_final_positions.pdb', 'w'))

<a id = 'smp'></a>
### Single Molecule Pulling

In [13]:
pull = CustomExternalForce('-fx*x-fy*y-fz*z')
system.addForce(pull)
pull.addPerParticleParameter('fx')
pull.addPerParticleParameter('fy')
pull.addPerParticleParameter('fz')

2

In [14]:
# Make sure that we have the right chain to pull
for ind, chain in enumerate(pdb.topology.chains()):
    if ind == 0:
        print(next(chain.atoms()))

<Atom 0 (N) of chain 0 residue 0 (ARG)>


In [15]:
res0 = next(pdb.topology.residues())
chain0 = next(pdb.topology.chains())
*_, last_at = chain0.atoms()
*_, last_res = chain0.residues()
res0_atoms = [x.index for x in res0.atoms()]
lastres_atoms =  [x.index for x in last_res.atoms()]

In [16]:
print(res0_atoms)
print(lastres_atoms)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887]


In [17]:
for x in res0_atoms:
    pull.addParticle(x,(-100, 0, 0)*kilojoules_per_mole/nanometer)
for x in lastres_atoms:
    pull.addParticle(x, (100, 0, 0)*kilojoules_per_mole/nanometer)

In [ ]:
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()

In [ ]:
simulation.reporters.append(PDBReporter('1QLQ_pulling.pdb', 2000))
simulation.reporters.append(StateDataReporter(stdout, 2000, step=True,
        potentialEnergy=True, temperature=True))

In [ ]:
simulation.step(20000)